In [ ]:
import numpy as np
import pandas as pd

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## SET folder to project directory
folder="C:\\evdoxiataka\\effi_user_study1\\"

In [ ]:
import os
from natsort import os_sorted

folder_path = folder+"data/collected_data/LOGS/"
files = []
for f in os.listdir(folder_path):
    fname = os.path.join(folder_path, f)
    if f[-4:] == '.csv':
        files.append(f)
files = os_sorted(files)

In [ ]:
feedback_df = pd.DataFrame()
for file_name in files:    
    # file_name = part_id+".csv"    
    file = pd.read_csv(folder+"data/collected_data/LOGs/'+file_name, delimiter=',')    
    # Conversion of App ID to int     
    file["App ID"] = file["App ID"].fillna(0.0).astype(int)
    # Reset_index 
    file.index = np.arange(0, len(file))    
    # Dropping Login
    file = file[file.Function != "Login"]
    file.reset_index(drop = True, inplace = True)  
    # Dropping Repeated Entries
    drop_index = []
    for i in range(1,len(file.index)):
        try:
            if ((file.at[i,"App ID"] == int(file.at[i,"Value"])) and (file.loc[i,"ID"] in file.loc[0:i-1,"ID"].unique()) and (file.loc[i,"App ID"]  in file.loc[0:i-1,"App ID"].unique())):          
                    drop_index.append(i)
        except: 
            continue    
    file.drop(index = drop_index,inplace = True)
    feedback_df = pd.concat([feedback_df,file],axis=0)    
feedback_df.drop(columns = "Pattern",inplace = True)# blank column
feedback_df.reset_index(drop = True, inplace = True)

In [ ]:
# Dropping the entires where function chosen is apply_refine_search which is filtering for attribute 
# but there is no attribute present that was filtered for. 
df_with_null_attribute = feedback_df[feedback_df["Attribute"].isna()]
null_attribute_index = df_with_null_attribute[df_with_null_attribute["Function"] == "apply_refine_search"].index
null_attribute_index

print(len(null_attribute_index))

feedback_df.drop(index = null_attribute_index,inplace = True)

feedback_df.reset_index(drop = True, inplace = True)
feedback_df.loc[15]

In [ ]:
similar_apps_ind = feedback_df[(feedback_df["Function"] == "similar_application") | (feedback_df["Function"] == "select_applications")].index

feedback_df.loc[similar_apps_ind, "Value"] = feedback_df.loc[similar_apps_ind, "Value"].astype(int)
type(feedback_df[feedback_df["Function"] == "similar_application"]["Value"][6])

In [ ]:
feedback_df.to_csv(folder+"data/processed_data/Feedback_final.csv", na_rep='None')